# Segmenting and Clustering Neighborhoods in Toronto
This notebook is used for the pear-graded assignment of the data science capstone course, week 3.

there are tree sub assignments to do. they will be marked by titles for quick reference:
    - assignment 1
    - assignment 2
    - assignment 3

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_420:

The following NEW packages will be INSTALLED:

    _libgcc_mutex: 0.1-main         conda-forge
    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.21.0-py_0      conda-forge
    readline:      7.0-ha6073c6_4              

The following packages will be UPDATED:

    conda:         4.3.31-py35_0                --> 4.5.11-py35_0        conda-forge
    pycosat:       0.6.1-py35_1                 --> 0.6.3-py35h470a237_1 conda-forge

The following packages will be SUPERSEDED by a higher-priority channel:

    conda-env:     2.6.0-h36134e3_1             --> 2.6.0-1              conda-forge

_libgcc_mutex- 100% |################################| Time: 0:00:00   1.66 MB/s
conda-env-2.6. 100% |################################| Time: 0:00:00 930.68 kB/s
readline-7.0-h 100% |################################| Time: 0:00:00  17.76 MB

In [ ]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## assignment 1
In this assignment we wil scrape date from the following wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and insert it in an pandas dataframe.

In [2]:
#first we need to install the BeautifulSoup package
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/nbuser/anaconda3_420

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2n             |       hb7f436b_0         3.4 MB  anaconda
    beautifulsoup4-4.6.3       |           py35_0         140 KB  anaconda
    ca-certificates-2017.08.26 |       h1d4fec5_0         263 KB  anaconda
    ------------------------------------------------------------
                                           Total:         3.8 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.5.1-py35_0          --> 4.6.3-py35_0          anaconda
    ca-certificates: 2017.08.26-h1d4fec5_0 --> 2017.08.26-h1d4f

To get the dataframe into pandas we will use beautifulsoup to scrape the website and turn it into a dataframe. 

In [23]:
from bs4 import BeautifulSoup
# Here, we're just importing Beautiful Soup
page_link = 'https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# this is the url.
page_response = requests.get(page_link, timeout=5)
# here, we fetch the content from the url, using the requests library
soup = BeautifulSoup(page_response.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.
table = soup.find_all('table')
df = pd.read_html(str(table), header=0)[0]

Now we got the raw table from the website into a pandas dataframe. the next step is to explore and clean the data.

In [24]:
print(df.shape)
df.head()

(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [25]:
#first we delete the rows where there is no Borough
# Get names of indexes for which column Borough is 'Not assigned'
indexNames = df[ df['Borough'] == 'Not assigned' ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In the database produced the Neighborhoods are already combined if they have the same Postal code. they are separated bij '/' instead of ',' so this will need to be replaced.

In [26]:
#Replace the ' /' by ','
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
print('Are there any duplicate values in Postal code:', df['Postal code'].duplicated().any())
df.describe()

Are there any duplicate values in Postal code: False


,Postal code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M5A,North York,Downsview
freq,1,24,4


As shown below there are no cells left in the dataframe with a NaN value.

In [28]:
df.isnull().sum(axis = 0)

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [31]:
df.shape

(103, 3)